In [1]:
from MODApy import vardb
from MODApy.vcfmgr import ParsedVCF
import pandas as pd
import duckdb

In [17]:
df = ParsedVCF.from_vcf('../../data/Patients/BM23-49617_MODApy/BM23-49617_MODApy.final.vcf')

In [3]:
df.name

'BM23-49617_MODApy'

In [7]:
variantsdb = vardb.VarDB('file','../../data/variantsDB/variantsDB.db')
variantsdb.conn.sql('SHOW ALL TABLES')

┌────────────┬─────────┬────────────┬──────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database  │ schema  │    name    │     column_names     │                 column_types                  │ temporary │
│  varchar   │ varchar │  varchar   │      varchar[]       │                   varchar[]                   │  boolean  │
├────────────┼─────────┼────────────┼──────────────────────┼───────────────────────────────────────────────┼───────────┤
│ variantsDB │ main    │ temp_varDB │ [CHROM, POS, REF, …  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR, VARCHAR…  │ false     │
│ variantsDB │ main    │ vars       │ [name, value]        │ [VARCHAR, VARCHAR]                            │ false     │
└────────────┴─────────┴────────────┴──────────────────────┴───────────────────────────────────────────────┴───────────┘

In [11]:
df_table_describe = variantsdb.conn.sql('DESCRIBE temp_varDB').to_df()

In [25]:
variantsdb.conn.sql("SELECT count(*) FROM information_schema.columns WHERE table_name = 'temp_varDB' LIMIT 100").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          511 │
└──────────────┘



In [16]:
'BM23-49619_MODApy' in df.column_name.to_list()

False

In [41]:
df.name = 'BM23-49619_MODApy'

In [42]:
sql = f'''WITH new_pat AS (SELECT CHROM,
                POS,
                REF,
                ALT,
                "HGVS.C",
                "HGVS.P",
                ZIGOSITY AS '{df.name}'
                FROM df)
            SELECT * FROM new_pat
            UNION BY NAME
            SELECT * from temp_varDB'''


In [50]:
list_before = variantsdb.conn.sql('SELECT * FROM temp_varDB').columns

In [49]:
list_after = variantsdb.conn.sql(sql).columns

In [51]:
'BM23-49619_MODApy' in list_before

False

In [52]:
'BM23-49619_MODApy' in list_after

True